In [1]:
!gdown 1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3

Downloading...
From: https://drive.google.com/uc?id=1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3
To: /content/Housing.csv
100% 30.0k/30.0k [00:00<00:00, 40.3MB/s]


In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor


In [17]:
data_path = "/content/Housing.csv"
df = pd.read_csv(data_path)

In [18]:
# quick review data
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [19]:
# NaN testing
df.isna().sum()

,0
price,0
area,0
bedrooms,0
bathrooms,0
stories,0
mainroad,0
guestroom,0
basement,0
hotwaterheating,0
airconditioning,0


In [20]:
categorical_cols = df.select_dtypes(include="object").columns.to_list()
print(categorical_cols)

['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']


In [21]:
ordinal_encoder = OrdinalEncoder()
df[categorical_cols] = ordinal_encoder.fit_transform(df[categorical_cols])
encoded_categorical_cols = ordinal_encoder.fit_transform(df[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical_cols, columns=categorical_cols)
numerical_df = df.drop(columns=categorical_cols, axis=1)
encoded_df = pd.concat([numerical_df, encoded_categorical_df], axis=1)

df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1.0,0.0,0.0,0.0,1.0,2,1.0,0.0
1,12250000,8960,4,4,4,1.0,0.0,0.0,0.0,1.0,3,0.0,0.0
2,12250000,9960,3,2,2,1.0,0.0,1.0,0.0,0.0,2,1.0,1.0
3,12215000,7500,4,2,2,1.0,0.0,1.0,0.0,1.0,3,1.0,0.0
4,11410000,7420,4,1,2,1.0,1.0,1.0,0.0,1.0,2,0.0,0.0


In [26]:
normalizer = StandardScaler()
dataset_arr = normalizer.fit_transform(encoded_df)
dataset_arr

array([[ 4.56636513,  1.04672629,  1.40341936, ...,  1.4726183 ,
         1.80494113, -1.40628573],
       [ 4.00448405,  1.75700953,  1.40341936, ...,  1.4726183 ,
        -0.55403469, -1.40628573],
       [ 4.00448405,  2.21823241,  0.04727831, ..., -0.67906259,
         1.80494113, -0.09166185],
       ...,
       [-1.61432675, -0.70592066, -1.30886273, ..., -0.67906259,
        -0.55403469,  1.22296203],
       [-1.61432675, -1.03338891,  0.04727831, ..., -0.67906259,
        -0.55403469, -1.40628573],
       [-1.61432675, -0.5998394 ,  0.04727831, ..., -0.67906259,
        -0.55403469,  1.22296203]])

In [27]:
X, y =dataset_arr [: , 1:] , dataset_arr [: , 0]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, shuffle=True
)

In [30]:
min_mse = None

for depth in range(1,11):
    model = DecisionTreeRegressor(max_depth=depth)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # checking each times
    print(f"Depth: {depth}, MSE: {mse}")

    if min_mse is None or mse < min_mse:
        min_mse = mse
        best_depth = depth

print(f"Best depth: {best_depth}, MSE: {min_mse}")

Depth: 1, MSE: 0.8104164672132036
Depth: 2, MSE: 0.6624731445240474
Depth: 3, MSE: 0.6226684506223363
Depth: 4, MSE: 0.5861341802065629
Depth: 5, MSE: 0.5556559601395509
Depth: 6, MSE: 0.5484162683734456
Depth: 7, MSE: 0.5989502525322169
Depth: 8, MSE: 0.6506423908824358
Depth: 9, MSE: 0.6279251056427427
Depth: 10, MSE: 0.682467241329469
Best depth: 6, MSE: 0.5484162683734456


In [31]:
random_forest_model = RandomForestRegressor(random_state=1)
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Random Forest MSE: {mse}")
mae = mean_absolute_error(y_test, y_pred)
print(f"Random Forest MAE: {mae}")

Random Forest MSE: 0.37944418523089524
Random Forest MAE: 0.46093873321571177


In [32]:
adaboost_model = AdaBoostRegressor(random_state=1)
adaboost_model.fit(X_train, y_train)
y_pred = adaboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"AdaBoost MSE: {mse}")
mae = mean_absolute_error(y_test, y_pred)
print(f"AdaBoost MAE: {mae}")

AdaBoost MSE: 0.5739244030038942
AdaBoost MAE: 0.567680019897059


In [33]:
gradient_boosting_model = GradientBoostingRegressor(random_state=1)
gradient_boosting_model.fit(X_train, y_train)
y_pred = gradient_boosting_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Gradient Boosting MSE: {mse}")
mae = mean_absolute_error(y_test, y_pred)
print(f"Gradient Boosting MAE: {mae}")

Gradient Boosting MSE: 0.39610445936979427
Gradient Boosting MAE: 0.4516626127750995
